<a href="https://colab.research.google.com/github/EML-Labs/Feature_Analysis/blob/main/Experiment_1_K2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [27]:
def compute_pc_l(R, l_max):
    """
    Compute p_c(l): probability of finding diagonals of length >= l
    R: recurrence matrix (NxN)
    l_max: maximum diagonal length to evaluate
    """
    N = R.shape[0]
    diag_lengths = []

    # extract diagonal line lengths

    # for k in range(0, N): # For Symmetric matrices
    for k in range(-N + 1, N):
        diag = np.diagonal(R, offset=k)
        print("Diagonal now:", diag)
        count = 0
        for val in diag:
            if val == 1:
                count += 1
            else:
                if count > 0:
                    diag_lengths.append(count)
                count = 0
        if count > 0:
            diag_lengths.append(count)

    diag_lengths = np.array(diag_lengths)
    print("Diagonal Lengths:", diag_lengths)

    pc = []
    for l in range(1, l_max + 1):
        pc.append(np.sum(diag_lengths >= l))

    pc = np.array(pc, dtype=float)
    pc /= pc[0]  # normalize by the diagonal
    return pc

# Unit Tests


In [32]:
#Checking pc_of_l functionality using random generator

# Synthetic recurrence matrix
np.random.seed(0)
N = 5
R = np.zeros((N, N))

for i in range(N):
    for j in range(N):
        if abs(i - j) < np.random.exponential(scale=10):
            R[i, j] = 1

# Step 1
l_max = 40
pc = compute_pc_l(R, l_max)
print("R:", R)
print("pc:", pc)


R: [[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 0.]
 [0. 0. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
pc: [1.  0.5 0.3 0.3 0.1 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
 0.  0.  0.  0. ]


In [29]:
# Checking pc_of_l functionality for symmetric matrix

R = [[1,0,1], [0,1,1], [1,1,1]]
R = np.array(R)
l_max = 5
pc = compute_pc_l(R, l_max)
print("R:", R)
print("pc:", pc)

Diagonal now: [1]
Diagonal now: [0 1]
Diagonal now: [1 1 1]
Diagonal now: [0 1]
Diagonal now: [1]
Diagonal Lengths: [1 1 3 1 1]
R: [[1 0 1]
 [0 1 1]
 [1 1 1]]
pc: [1.  0.2 0.2 0.  0. ]
